# সেশন ৪ – SLM বনাম LLM তুলনা

একটি ছোট ভাষা মডেল এবং Foundry Local-এ চলমান একটি বড় মডেলের মধ্যে লেটেন্সি এবং নমুনা প্রতিক্রিয়া গুণমান তুলনা করুন।


## ⚡ দ্রুত শুরু

**মেমোরি-অপ্টিমাইজড সেটআপ (আপডেট):**
1. মডেলগুলো স্বয়ংক্রিয়ভাবে CPU ভ্যারিয়েন্ট নির্বাচন করে (যেকোনো হার্ডওয়্যারে কাজ করে)
2. `qwen2.5-3b` ব্যবহার করে 7B এর পরিবর্তে (প্রায় ~4GB RAM সাশ্রয় করে)
3. পোর্ট স্বয়ংক্রিয়ভাবে সনাক্ত হয় (কোনো ম্যানুয়াল কনফিগারেশন প্রয়োজন নেই)
4. মোট প্রয়োজনীয় RAM: ~8GB সুপারিশকৃত (মডেল + OS)

**টার্মিনাল সেটআপ (৩০ সেকেন্ড):**
```bash
foundry service start
foundry model run phi-4-mini
foundry model run qwen2.5-3b
```

তারপর এই নোটবুকটি চালান! 🚀


### ব্যাখ্যা: নির্ভরতা ইনস্টলেশন
টাইমিং এবং চ্যাট অনুরোধের জন্য প্রয়োজনীয় ন্যূনতম প্যাকেজ (`foundry-local-sdk`, `openai`, `numpy`) ইনস্টল করে। এটি নিরাপদভাবে পুনরায় চালানো যেতে পারে।


# পরিস্থিতি
একটি ছোট ভাষা মডেল (SLM) এবং একটি বড় মডেলের মধ্যে একটি প্রম্পটের ভিত্তিতে তুলনা করুন, যাতে সুবিধা এবং সীমাবদ্ধতাগুলি বোঝা যায়:
- **লেটেন্সি পার্থক্য** (ওয়াল ক্লক সেকেন্ড)
- **টোকেন ব্যবহার** (যদি উপলব্ধ থাকে) থ্রুপুটের সূচক হিসেবে
- **গুণগত আউটপুটের নমুনা** দ্রুত পর্যবেক্ষণের জন্য
- **গতি বৃদ্ধির হিসাব** কর্মক্ষমতা উন্নতির পরিমাণ নির্ধারণ করতে

**পরিবেশ ভেরিয়েবল:**
- `SLM_ALIAS` - ছোট ভাষা মডেল (ডিফল্ট: phi-4-mini, ~4GB RAM)
- `LLM_ALIAS` - বড় ভাষা মডেল (ডিফল্ট: qwen2.5-7b, ~7GB RAM)
- `COMPARE_PROMPT` - তুলনার জন্য পরীক্ষার প্রম্পট
- `COMPARE_RETRIES` - স্থিতিশীলতার জন্য পুনরায় চেষ্টা করার সংখ্যা (ডিফল্ট: 2)
- `FOUNDRY_LOCAL_ENDPOINT` - সার্ভিস এন্ডপয়েন্ট ওভাররাইড করুন (যদি সেট না করা থাকে, স্বয়ংক্রিয়ভাবে সনাক্ত করা হবে)

**কিভাবে কাজ করে (অফিশিয়াল SDK প্যাটার্ন):**
1. **FoundryLocalManager** Foundry Local সার্ভিসটি আরম্ভ এবং পরিচালনা করে
2. সার্ভিসটি স্বয়ংক্রিয়ভাবে শুরু হয় যদি এটি চালু না থাকে (কোনো ম্যানুয়াল সেটআপের প্রয়োজন নেই)
3. মডেলগুলি অ্যালিয়াস থেকে স্বয়ংক্রিয়ভাবে নির্দিষ্ট আইডিতে রূপান্তরিত হয়
4. হার্ডওয়্যার-অপ্টিমাইজড ভ্যারিয়েন্ট নির্বাচন করা হয় (CUDA, NPU, বা CPU)
5. OpenAI-সামঞ্জস্যপূর্ণ ক্লায়েন্ট চ্যাট কমপ্লিশন সম্পন্ন করে
6. মেট্রিক্স সংগ্রহ করা হয়: লেটেন্সি, টোকেন, আউটপুট গুণমান
7. ফলাফল তুলনা করে গতি বৃদ্ধির অনুপাত গণনা করা হয়

এই ক্ষুদ্র-তুলনা আপনাকে সিদ্ধান্ত নিতে সাহায্য করে যে আপনার ব্যবহারের ক্ষেত্রে বড় মডেলের দিকে রাউটিং কখন যুক্তিযুক্ত।

**SDK রেফারেন্স:** 
- পাইথন SDK: https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local
- ওয়ার্কশপ ইউটিলস: ../samples/workshop_utils.py থেকে অফিসিয়াল প্যাটার্ন ব্যবহার করে

**মূল সুবিধাসমূহ:**
- ✅ স্বয়ংক্রিয় সার্ভিস সনাক্তকরণ এবং আরম্ভ
- ✅ সার্ভিস চালু না থাকলে স্বয়ংক্রিয়ভাবে শুরু
- ✅ বিল্ট-ইন মডেল রেজোলিউশন এবং ক্যাশিং
- ✅ হার্ডওয়্যার অপ্টিমাইজেশন (CUDA/NPU/CPU)
- ✅ OpenAI SDK সামঞ্জস্যপূর্ণ
- ✅ পুনরায় চেষ্টা সহ শক্তিশালী ত্রুটি পরিচালনা
- ✅ স্থানীয় ইনফারেন্স (ক্লাউড API প্রয়োজন নেই)


## 🚨 প্রয়োজনীয়তা: Foundry Local চালু থাকতে হবে!

**এই নোটবুক চালানোর আগে**, নিশ্চিত করুন যে Foundry Local সার্ভিস সেটআপ করা হয়েছে:

### দ্রুত শুরু কমান্ড (টার্মিনালে চালান):

```bash
# 1. Start the Foundry Local service
foundry service start

# 2. Load the default models used in this comparison (CPU-optimized)
foundry model run phi-4-mini
foundry model run qwen2.5-3b

# 3. Verify models are loaded
foundry model ls

# 4. Check service health
foundry service status
```

### বিকল্প মডেল (যদি ডিফল্ট উপলব্ধ না থাকে):

```bash
# Even smaller alternatives (if memory is very limited)
foundry model run phi-3.5-mini
foundry model run qwen2.5-0.5b

# Or update the environment variables in this notebook:
# SLM_ALIAS = 'phi-3.5-mini'
# LLM_ALIAS = 'qwen2.5-1.5b'  # Or qwen2.5-0.5b for minimum memory
```

⚠️ **যদি আপনি এই ধাপগুলি এড়িয়ে যান**, তাহলে নীচের নোটবুক সেল চালানোর সময় `APIConnectionError` দেখতে পাবেন।


In [29]:
# Install dependencies
!pip install -q foundry-local-sdk openai numpy requests

### ব্যাখ্যা: মূল আমদানি
টাইমিং ইউটিলিটি এবং Foundry Local / OpenAI ক্লায়েন্ট নিয়ে আসে যা মডেলের তথ্য আনতে এবং চ্যাট সম্পন্ন করতে ব্যবহৃত হয়।


In [30]:
import os, time, json
from foundry_local import FoundryLocalManager
from openai import OpenAI
import sys
sys.path.append('../samples')
from workshop_utils import get_client, chat_once

### ব্যাখ্যা: এলিয়াস এবং প্রম্পট সেটআপ
ছোট এবং বড় মডেলের জন্য পরিবেশ-নিয়ন্ত্রিত এলিয়াস এবং একটি তুলনামূলক প্রম্পট নির্ধারণ করে। বিভিন্ন মডেল পরিবার বা কাজের সাথে পরীক্ষা করার জন্য পরিবেশ ভেরিয়েবলগুলি সামঞ্জস্য করুন।


In [31]:
# Default to CPU models for better memory efficiency
SLM = os.getenv('SLM_ALIAS', 'phi-4-mini')  # Auto-selects CPU variant
LLM = os.getenv('LLM_ALIAS', 'qwen2.5-3b')  # Smaller LLM, more memory-friendly
PROMPT = os.getenv('COMPARE_PROMPT', 'List 5 benefits of local AI inference.')
# Endpoint is now managed by FoundryLocalManager - it auto-detects or can be overridden
ENDPOINT = os.getenv('FOUNDRY_LOCAL_ENDPOINT', None)

### 💡 মেমোরি-অপ্টিমাইজড কনফিগারেশন

**এই নোটবুকটি ডিফল্টভাবে মেমোরি-সাশ্রয়ী মডেল ব্যবহার করে:**
- `phi-4-mini` → ~4GB RAM (Foundry Local স্বয়ংক্রিয়ভাবে CPU ভ্যারিয়েন্ট নির্বাচন করে)
- `qwen2.5-3b` → ~3GB RAM (7B এর পরিবর্তে, যা ~7GB+ প্রয়োজন)

**পোর্ট স্বয়ংক্রিয় সনাক্তকরণ:**
- Foundry Local বিভিন্ন পোর্ট ব্যবহার করতে পারে (সাধারণত 55769 বা 59959)
- নিচের ডায়াগনস্টিক সেলটি সঠিক পোর্ট স্বয়ংক্রিয়ভাবে সনাক্ত করে
- কোনো ম্যানুয়াল কনফিগারেশন প্রয়োজন নেই!

**যদি আপনার RAM সীমিত হয় (<8GB), আরও ছোট মডেল ব্যবহার করুন:**
```python
SLM = 'phi-3.5-mini'      # ~2GB
LLM = 'qwen2.5-0.5b'      # ~500MB
```


In [32]:
# Display current configuration
print("="*60)
print("CURRENT CONFIGURATION")
print("="*60)
print(f"SLM Model:     {SLM}")
print(f"LLM Model:     {LLM}")
print(f"SDK Pattern:   FoundryLocalManager (official)")
print(f"Endpoint:      {ENDPOINT or 'Auto-detect'}")
print(f"Test Prompt:   {PROMPT[:50]}...")
print(f"Retry Count:   2")
print("="*60)
print("\n💡 Using official Foundry SDK pattern from workshop_utils")
print("   → FoundryLocalManager handles service lifecycle")
print("   → Automatic model resolution and hardware optimization")
print("   → OpenAI-compatible API for inference")

CURRENT CONFIGURATION
SLM Model:     phi-4-mini
LLM Model:     qwen2.5-7b
SDK Pattern:   FoundryLocalManager (official)
Endpoint:      Auto-detect
Test Prompt:   List 5 benefits of local AI inference....
Retry Count:   2

💡 Using official Foundry SDK pattern from workshop_utils
   → FoundryLocalManager handles service lifecycle
   → Automatic model resolution and hardware optimization
   → OpenAI-compatible API for inference


### ব্যাখ্যা: এক্সিকিউশন হেল্পারস (Foundry SDK প্যাটার্ন)
ওয়ার্কশপের নমুনাগুলিতে নথিভুক্ত অফিসিয়াল Foundry Local SDK প্যাটার্ন ব্যবহার করে:

**পদ্ধতি:**
- **FoundryLocalManager** - Foundry Local সার্ভিসটি আরম্ভ এবং পরিচালনা করে
- **অটো-ডিটেকশন** - স্বয়ংক্রিয়ভাবে এন্ডপয়েন্ট আবিষ্কার করে এবং সার্ভিস লাইফসাইকেল পরিচালনা করে
- **মডেল রেজোলিউশন** - উপনামগুলিকে পূর্ণ মডেল আইডিতে রূপান্তর করে (যেমন, phi-4-mini → phi-4-mini-instruct-cpu)
- **হার্ডওয়্যার অপ্টিমাইজেশন** - উপলব্ধ হার্ডওয়্যারের জন্য সেরা ভ্যারিয়েন্ট নির্বাচন করে (CUDA, NPU, বা CPU)
- **OpenAI ক্লায়েন্ট** - OpenAI-সামঞ্জস্যপূর্ণ API অ্যাক্সেসের জন্য ম্যানেজারের এন্ডপয়েন্ট দিয়ে কনফিগার করা

**সহনশীলতার বৈশিষ্ট্য:**
- এক্সপোনেনশিয়াল ব্যাকঅফ রিট্রাই লজিক (পরিবেশের মাধ্যমে কনফিগারযোগ্য)
- সার্ভিস চালু না থাকলে স্বয়ংক্রিয়ভাবে আরম্ভ করা
- আরম্ভের পরে সংযোগ যাচাই
- বিশদ ত্রুটি রিপোর্টিং সহ নমনীয় ত্রুটি পরিচালনা
- পুনরাবৃত্তি আরম্ভ এড়াতে মডেল ক্যাশিং

**ফলাফলের কাঠামো:**
- লেটেন্সি পরিমাপ (ওয়াল ক্লক টাইম)
- টোকেন ব্যবহারের ট্র্যাকিং (যদি উপলব্ধ থাকে)
- নমুনা আউটপুট (পাঠযোগ্যতার জন্য সংক্ষিপ্ত)
- ব্যর্থ অনুরোধের জন্য ত্রুটির বিশদ

এই প্যাটার্নটি workshop_utils মডিউল ব্যবহার করে, যা অফিসিয়াল SDK প্যাটার্ন অনুসরণ করে।

**SDK রেফারেন্স:**
- প্রধান রিপোজিটরি: https://github.com/microsoft/Foundry-Local
- পাইথন SDK: https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local
- ওয়ার্কশপ ইউটিলস: ../samples/workshop_utils.py


In [39]:
def setup(alias: str, endpoint: str = None, retries: int = 3):
    """
    Initialize a Foundry Local model connection using official SDK pattern.
    
    This follows the workshop_utils pattern which uses FoundryLocalManager
    to properly initialize the Foundry Local service and resolve models.
    
    Args:
        alias: Model alias (e.g., 'phi-4-mini', 'qwen2.5-3b')
        endpoint: Optional endpoint override (usually auto-detected)
        retries: Number of connection attempts (default: 3)
    
    Returns:
        tuple: (manager, client, model_id, metadata) or (None, None, alias, error_metadata) if failed
    """
    import time
    
    last_err = None
    current_delay = 2  # seconds
    
    for attempt in range(1, retries + 1):
        try:
            print(f"[Init] Connecting to '{alias}' (attempt {attempt}/{retries})...")
            
            # Use the workshop utility which follows the official SDK pattern
            manager, client, model_id = get_client(alias, endpoint=endpoint)
            
            print(f"[OK] Connected to '{alias}' -> {model_id}")
            print(f"     Endpoint: {manager.endpoint}")
            
            return manager, client, model_id, {
                'endpoint': manager.endpoint,
                'resolved': model_id,
                'attempts': attempt,
                'status': 'success'
            }
            
        except Exception as e:
            last_err = e
            error_msg = str(e)
            
            # Provide helpful error messages
            if "Connection error" in error_msg or "connection refused" in error_msg.lower():
                print(f"[ERROR] Cannot connect to Foundry Local service")
                print(f"        → Is the service running? Try: foundry service start")
                print(f"        → Is the model loaded? Try: foundry model run {alias}")
            elif "not found" in error_msg.lower():
                print(f"[ERROR] Model '{alias}' not found in catalog")
                print(f"        → Available models: Run 'foundry model ls' in terminal")
                print(f"        → Download model: Run 'foundry model download {alias}'")
            else:
                print(f"[ERROR] Setup failed: {type(e).__name__}: {error_msg}")
            
            if attempt < retries:
                print(f"[Retry] Waiting {current_delay:.1f}s before retry...")
                time.sleep(current_delay)
                current_delay *= 2  # Exponential backoff
    
    # All retries failed - provide actionable guidance
    print(f"\n❌ Failed to initialize '{alias}' after {retries} attempts")
    print(f"   Last error: {type(last_err).__name__}: {str(last_err)}")
    print(f"\n💡 Troubleshooting steps:")
    print(f"   1. Ensure Foundry Local service is running:")
    print(f"      → foundry service status")
    print(f"      → foundry service start (if not running)")
    print(f"   2. Ensure model is loaded:")
    print(f"      → foundry model run {alias}")
    print(f"   3. Check available models:")
    print(f"      → foundry model ls")
    print(f"   4. Try alternative models if '{alias}' isn't available")
    
    return None, None, alias, {
        'error': f"{type(last_err).__name__}: {str(last_err)}",
        'endpoint': endpoint or 'auto-detect',
        'attempts': retries,
        'status': 'failed'
    }


def run(client, model_id: str, prompt: str, max_tokens: int = 180, temperature: float = 0.5):
    """
    Run inference with the configured model using OpenAI SDK.
    
    Args:
        client: OpenAI client instance (configured for Foundry Local)
        model_id: Model identifier (resolved from alias)
        prompt: Input prompt
        max_tokens: Maximum response tokens (default: 180)
        temperature: Sampling temperature (default: 0.5)
    
    Returns:
        dict: Response with timing, tokens, and content
    """
    import time
    
    start = time.time()
    
    try:
        response = client.chat.completions.create(
            model=model_id,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            temperature=temperature
        )
        
        elapsed = time.time() - start
        
        # Extract response details
        content = response.choices[0].message.content
        
        # Try to extract token usage from multiple possible locations
        usage_info = {}
        if hasattr(response, 'usage') and response.usage:
            usage_info['prompt_tokens'] = getattr(response.usage, 'prompt_tokens', None)
            usage_info['completion_tokens'] = getattr(response.usage, 'completion_tokens', None)
            usage_info['total_tokens'] = getattr(response.usage, 'total_tokens', None)
        
        # Calculate approximate token count if API doesn't provide it
        # Rough estimate: ~4 characters per token for English text
        if not usage_info.get('total_tokens'):
            estimated_prompt_tokens = len(prompt) // 4
            estimated_completion_tokens = len(content) // 4
            estimated_total = estimated_prompt_tokens + estimated_completion_tokens
            usage_info['estimated_tokens'] = estimated_total
            usage_info['estimated_prompt_tokens'] = estimated_prompt_tokens
            usage_info['estimated_completion_tokens'] = estimated_completion_tokens
        
        return {
            'status': 'success',
            'content': content,
            'elapsed_sec': elapsed,
            'tokens': usage_info.get('total_tokens') or usage_info.get('estimated_tokens'),
            'usage': usage_info,
            'model': model_id
        }
        
    except Exception as e:
        elapsed = time.time() - start
        return {
            'status': 'error',
            'error': f"{type(e).__name__}: {str(e)}",
            'elapsed_sec': elapsed,
            'model': model_id
        }


print("✅ Execution helpers defined: setup(), run()")
print("   → Uses workshop_utils for proper SDK integration")
print("   → setup() initializes with FoundryLocalManager")
print("   → run() executes inference via OpenAI-compatible API")
print("   → Token counting: Uses API data or estimates if unavailable")

✅ Execution helpers defined: setup(), run()
   → Uses workshop_utils for proper SDK integration
   → setup() initializes with FoundryLocalManager
   → run() executes inference via OpenAI-compatible API
   → Token counting: Uses API data or estimates if unavailable


### ব্যাখ্যা: প্রি-ফ্লাইট সেলফ-টেস্ট
FoundryLocalManager ব্যবহার করে উভয় মডেলের জন্য একটি হালকা কানেক্টিভিটি চেক চালানো হয়। এটি নিশ্চিত করে:
- সার্ভিস অ্যাক্সেসযোগ্য
- মডেলগুলো ইনিশিয়ালাইজ করা যায়
- অ্যালিয়াসগুলো প্রকৃত মডেল আইডিতে রেজল্ভ হয়
- তুলনা চালানোর আগে সংযোগ স্থিতিশীল

setup() ফাংশন workshop_utils থেকে অফিসিয়াল SDK প্যাটার্ন ব্যবহার করে।


In [34]:
# Simplified diagnostic: Just verify service is accessible
import requests

def check_foundry_service():
    """Quick diagnostic to verify Foundry Local is running."""
    # Try common ports
    endpoints_to_try = [
        "http://localhost:59959",
        "http://127.0.0.1:59959", 
        "http://localhost:55769",
        "http://127.0.0.1:55769",
    ]
    
    print("[Diagnostic] Checking Foundry Local service...")
    
    for endpoint in endpoints_to_try:
        try:
            response = requests.get(f"{endpoint}/health", timeout=2)
            if response.status_code == 200:
                print(f"✅ Service is running at {endpoint}")
                
                # Try to list models
                try:
                    models_response = requests.get(f"{endpoint}/v1/models", timeout=2)
                    if models_response.status_code == 200:
                        models_data = models_response.json()
                        model_count = len(models_data.get('data', []))
                        print(f"✅ Found {model_count} models available")
                        if model_count > 0:
                            print("   Models:", [m.get('id', 'unknown') for m in models_data.get('data', [])[:5]])
                except Exception as e:
                    print(f"⚠️  Could not list models: {e}")
                
                return endpoint
        except requests.exceptions.ConnectionError:
            continue
        except Exception as e:
            print(f"⚠️  Error checking {endpoint}: {e}")
    
    print("\n❌ Foundry Local service not found!")
    print("\n💡 To fix this:")
    print("   1. Open a terminal")
    print("   2. Run: foundry service start")
    print("   3. Run: foundry model run phi-4-mini")
    print("   4. Run: foundry model run qwen2.5-3b")
    print("   5. Re-run this notebook")
    return None

# Run diagnostic
discovered_endpoint = check_foundry_service()

if discovered_endpoint:
    print(f"\n✅ Service detected (will be managed by FoundryLocalManager)")
else:
    print(f"\n⚠️  No service detected - FoundryLocalManager will attempt to start it")

[Diagnostic] Checking Foundry Local service...

❌ Foundry Local service not found!

💡 To fix this:
   1. Open a terminal
   2. Run: foundry service start
   3. Run: foundry model run phi-4-mini
   4. Run: foundry model run qwen2.5-3b
   5. Re-run this notebook

⚠️  No service detected - FoundryLocalManager will attempt to start it


In [35]:
# Quick Fix: Start service and load models from notebook
# Uncomment the commands you need:

# !foundry service start
# !foundry model run phi-4-mini
# !foundry model run qwen2.5-3b
# !foundry model ls

print("⚠️  The commands above are commented out.")
print("Uncomment them if you want to start the service from the notebook.")
print("")
print("💡 Recommended: Run these commands in a separate terminal instead:")
print("   foundry service start")
print("   foundry model run phi-4-mini")
print("   foundry model run qwen2.5-3b")

⚠️  The commands above are commented out.
Uncomment them if you want to start the service from the notebook.

💡 Recommended: Run these commands in a separate terminal instead:
   foundry service start
   foundry model run phi-4-mini
   foundry model run qwen2.5-3b


### 🛠️ দ্রুত সমাধান: নোটবুক থেকে Foundry Local শুরু করুন (ঐচ্ছিক)

যদি উপরের ডায়াগনস্টিক দেখায় যে সার্ভিস চালু নেই, আপনি এটি এখান থেকে শুরু করার চেষ্টা করতে পারেন:

**নোট:** এটি উইন্ডোজে সবচেয়ে ভালো কাজ করে। অন্যান্য প্ল্যাটফর্মে, টার্মিনাল কমান্ড ব্যবহার করুন।


### ⚠️ সংযোগ ত্রুটি সমাধান

যদি আপনি `APIConnectionError` দেখতে পান, তাহলে সম্ভবত Foundry Local সার্ভিস চালু নেই বা মডেলগুলো লোড করা হয়নি। এই ধাপগুলো চেষ্টা করুন:

**১. সার্ভিসের স্ট্যাটাস পরীক্ষা করুন:**
```bash
# In a terminal (not in notebook):
foundry service status
```

**২. সার্ভিস চালু করুন (যদি চালু না থাকে):**
```bash
foundry service start
```

**৩. প্রয়োজনীয় মডেলগুলো লোড করুন:**
```bash
# Load the models needed for comparison
foundry model run phi-4-mini
foundry model run qwen2.5-7b

# Or use alternative models:
foundry model run phi-3.5-mini
foundry model run qwen2.5-3b
```

**৪. নিশ্চিত করুন মডেলগুলো উপলব্ধ আছে:**
```bash
foundry model ls
```

**সাধারণ সমস্যা:**
- ❌ সার্ভিস চালু নেই → `foundry service start` চালান
- ❌ মডেলগুলো লোড করা হয়নি → `foundry model run <model-name>` চালান
- ❌ পোর্টের সংঘর্ষ → দেখুন অন্য কোনো সার্ভিস পোর্ট ব্যবহার করছে কিনা
- ❌ ফায়ারওয়াল ব্লক করছে → নিশ্চিত করুন লোকাল সংযোগগুলো অনুমোদিত

**দ্রুত সমাধান:** প্রি-ফ্লাইট চেকের আগে নিচের ডায়াগনস্টিক সেলটি চালান।


In [36]:
preflight = {}
retries = 2  # Number of retry attempts

for a in (SLM, LLM):
    mgr, c, mid, info = setup(a, endpoint=ENDPOINT, retries=retries)
    # Keep the original status from info (either 'success' or 'failed')
    preflight[a] = info

print('\n[Pre-flight Check]')
for alias, details in preflight.items():
    status_icon = '✅' if details['status'] == 'success' else '❌'
    print(f"  {status_icon} {alias}: {details['status']} - {details.get('resolved', details.get('error', 'unknown'))}")

preflight

[Init] Connecting to 'phi-4-mini' (attempt 1/2)...
[OK] Connected to 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1

[Pre-flight Check]
  ✅ phi-4-mini: success - Phi-4-mini-instruct-cuda-gpu:4
  ✅ qwen2.5-7b: success - qwen2.5-7b-instruct-cuda-gpu:3


{'phi-4-mini': {'endpoint': 'http://127.0.0.1:59959/v1',
  'resolved': 'Phi-4-mini-instruct-cuda-gpu:4',
  'attempts': 1,
  'status': 'success'},
 'qwen2.5-7b': {'endpoint': 'http://127.0.0.1:59959/v1',
  'resolved': 'qwen2.5-7b-instruct-cuda-gpu:3',
  'attempts': 1,
  'status': 'success'}}

### ✅ প্রি-ফ্লাইট চেক: মডেলের প্রাপ্যতা

এই সেলটি নিশ্চিত করে যে উভয় মডেল নির্ধারিত এন্ডপয়েন্টে পৌঁছানো যায় কিনা, তুলনা চালানোর আগে।


### ব্যাখ্যা: রান তুলনা এবং ফলাফল সংগ্রহ
অফিশিয়াল Foundry SDK প্যাটার্ন ব্যবহার করে উভয় এলিয়াসের উপর পুনরাবৃত্তি করে:
1. প্রতিটি মডেলকে setup() দিয়ে ইনিশিয়ালাইজ করুন (FoundryLocalManager ব্যবহার করে)
2. OpenAI-সামঞ্জস্যপূর্ণ API দিয়ে ইনফারেন্স চালান
3. লেটেন্সি, টোকেন এবং নমুনা আউটপুট সংগ্রহ করুন
4. তুলনামূলক বিশ্লেষণ সহ JSON সারাংশ তৈরি করুন

এটি session04/model_compare.py-তে Workshop নমুনাগুলির একই প্যাটার্ন অনুসরণ করে।


In [40]:
results = []
retries = 2  # Number of retry attempts

for alias in (SLM, LLM):
    mgr, client, mid, info = setup(alias, endpoint=ENDPOINT, retries=retries)
    if client:
        r = run(client, mid, PROMPT)
        results.append({'alias': alias, **r})
    else:
        # If setup failed, record error
        results.append({
            'alias': alias,
            'status': 'error',
            'error': info.get('error', 'Setup failed'),
            'elapsed_sec': 0,
            'tokens': None,
            'model': alias
        })

# Display results
print(json.dumps(results, indent=2))

# Quick comparative view
print('\n' + '='*80)
print('COMPARISON SUMMARY')
print('='*80)
print(f"{'Alias':<20} {'Status':<15} {'Latency(s)':<15} {'Tokens':<15}")
print('-'*80)

for row in results:
    status = row.get('status', 'unknown')
    status_icon = '✅' if status == 'success' else '❌'
    latency_str = f"{row.get('elapsed_sec', 0):.3f}" if row.get('elapsed_sec') else 'N/A'
    
    # Handle token display - show if available or indicate estimated
    tokens = row.get('tokens')
    usage = row.get('usage', {})
    if tokens:
        if 'estimated_tokens' in usage:
            tokens_str = f"~{tokens} (est.)"
        else:
            tokens_str = str(tokens)
    else:
        tokens_str = 'N/A'
    
    print(f"{status_icon} {row['alias']:<18} {status:<15} {latency_str:<15} {tokens_str:<15}")

print('-'*80)

# Show detailed token breakdown if available
print("\nDetailed Token Usage:")
for row in results:
    if row.get('status') == 'success' and row.get('usage'):
        usage = row['usage']
        print(f"\n  {row['alias']}:")
        if 'prompt_tokens' in usage and usage['prompt_tokens']:
            print(f"    Prompt tokens:     {usage['prompt_tokens']}")
            print(f"    Completion tokens: {usage['completion_tokens']}")
            print(f"    Total tokens:      {usage['total_tokens']}")
        elif 'estimated_tokens' in usage:
            print(f"    Estimated prompt:     {usage['estimated_prompt_tokens']}")
            print(f"    Estimated completion: {usage['estimated_completion_tokens']}")
            print(f"    Estimated total:      {usage['estimated_tokens']}")
            print(f"    (API did not provide token counts - using ~4 chars/token estimate)")

print('\n' + '='*80)

# Calculate speedup if both succeeded
if len(results) == 2 and all(r.get('status') == 'success' and r.get('elapsed_sec') for r in results):
    speedup = results[1]['elapsed_sec'] / results[0]['elapsed_sec']
    print(f"\n💡 SLM is {speedup:.2f}x faster than LLM for this prompt")
    
    # Compare token throughput if available
    slm_tokens = results[0].get('tokens', 0)
    llm_tokens = results[1].get('tokens', 0)
    if slm_tokens and llm_tokens:
        slm_tps = slm_tokens / results[0]['elapsed_sec']
        llm_tps = llm_tokens / results[1]['elapsed_sec']
        print(f"   SLM throughput: {slm_tps:.1f} tokens/sec")
        print(f"   LLM throughput: {llm_tps:.1f} tokens/sec")
        
elif any(r.get('status') == 'error' for r in results):
    print(f"\n⚠️  Some models failed - check errors above")
    print("   Ensure Foundry Local is running: foundry service start")
    print("   Ensure models are loaded: foundry model run <model-name>")

results

[Init] Connecting to 'phi-4-mini' (attempt 1/2)...
[OK] Connected to 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1
[
  {
    "alias": "phi-4-mini",
    "status": "success",
    "content": "1. Reduced Latency: Local AI inference can significantly reduce latency by processing data closer to the source, which is particularly beneficial for real-time applications such as autonomous vehicles or augmented reality.\n\n2. Enhanced Privacy: By keeping data processing local, sensitive information is less likely to be exposed to external networks, thereby enhancing privacy and security.\n\n3. Lower Bandwidth Usage: Local AI inference reduces the n

[{'alias': 'phi-4-mini',
  'status': 'success',
  'content': '1. Reduced Latency: Local AI inference can significantly reduce latency by processing data closer to the source, which is particularly beneficial for real-time applications such as autonomous vehicles or augmented reality.\n\n2. Enhanced Privacy: By keeping data processing local, sensitive information is less likely to be exposed to external networks, thereby enhancing privacy and security.\n\n3. Lower Bandwidth Usage: Local AI inference reduces the need for data transmission over the network, which can save bandwidth and reduce the risk of network congestion.\n\n4. Improved Reliability: Local processing can be more reliable, as it is less dependent on network connectivity. This is particularly important in scenarios where network connectivity is unreliable or intermittent.\n\n5. Scalability: Local AI inference can be easily scaled by adding more local processing units, making it easier to handle increasing data volumes or m

### ফলাফল বিশ্লেষণ

**মূল সূচকসমূহ:**
- **লেটেন্সি**: কম হলে ভালো - দ্রুত প্রতিক্রিয়া সময় নির্দেশ করে
- **টোকেন**: বেশি থ্রুপুট = বেশি টোকেন প্রক্রিয়াকৃত
- **রুট**: নিশ্চিত করে কোন API এন্ডপয়েন্ট ব্যবহার করা হয়েছে

**SLM বনাম LLM কখন ব্যবহার করবেন:**
- **SLM (ছোট ভাষা মডেল)**: দ্রুত প্রতিক্রিয়া, কম রিসোর্স ব্যবহার, সহজ কাজের জন্য উপযুক্ত
- **LLM (বড় ভাষা মডেল)**: উচ্চ মানের, ভালো যুক্তি প্রদান, যখন মান সবচেয়ে গুরুত্বপূর্ণ তখন ব্যবহার করুন

**পরবর্তী পদক্ষেপ:**
1. বিভিন্ন প্রম্পট চেষ্টা করুন এবং দেখুন কিভাবে জটিলতা তুলনার উপর প্রভাব ফেলে
2. অন্যান্য মডেল জোড়া নিয়ে পরীক্ষা করুন
3. Workshop রাউটার নমুনা (Session 06) ব্যবহার করুন কাজের জটিলতার উপর ভিত্তি করে বুদ্ধিমত্তার সাথে রাউট করার জন্য


In [38]:
# Final Validation Check
print("="*70)
print("VALIDATION SUMMARY")
print("="*70)
print(f"✅ SLM Model: {SLM}")
print(f"✅ LLM Model: {LLM}")
print(f"✅ Using Foundry SDK Pattern: workshop_utils with FoundryLocalManager")
print(f"✅ Pre-flight passed: {all(v['status'] == 'success' for v in preflight.values()) if 'preflight' in dir() else 'Not run yet'}")
print(f"✅ Comparison completed: {len(results) == 2 if 'results' in dir() else 'Not run yet'}")
print(f"✅ Both models responded: {all(r.get('status') == 'success' for r in results) if 'results' in dir() and results else 'Not run yet'}")
print("="*70)

# Check for common configuration issues
issues = []
if 'LLM' in dir() and LLM not in ['qwen2.5-3b', 'qwen2.5-0.5b', 'qwen2.5-1.5b', 'qwen2.5-7b', 'phi-3.5-mini']:
    issues.append(f"⚠️  LLM is '{LLM}' - expected qwen2.5-3b for memory efficiency")
if 'preflight' in dir() and not all(v['status'] == 'success' for v in preflight.values()):
    issues.append("⚠️  Pre-flight check failed - models not accessible")
if 'results' in dir() and results and not all(r.get('status') == 'success' for r in results):
    issues.append("⚠️  Comparison incomplete - check for errors above")

if not issues and 'results' in dir() and results and all(r.get('status') == 'success' for r in results):
    print("🎉 ALL CHECKS PASSED! Notebook completed successfully.")
    print(f"   SLM ({SLM}) vs LLM ({LLM}) comparison completed.")
    if len(results) == 2:
        speedup = results[1]['elapsed_sec'] / results[0]['elapsed_sec'] if results[0]['elapsed_sec'] > 0 else 0
        print(f"   Performance: SLM is {speedup:.2f}x faster")
elif issues:
    print("\n⚠️  Issues detected:")
    for issue in issues:
        print(f"   {issue}")
    print("\n💡 Troubleshooting:")
    print("   1. Ensure service is running: foundry service start")
    print("   2. Load models: foundry model run phi-4-mini && foundry model run qwen2.5-7b")
    print("   3. Check model list: foundry model ls")
else:
    print("\n💡 Run all cells above first, then re-run this validation.")
print("="*70)

VALIDATION SUMMARY
✅ SLM Model: phi-4-mini
✅ LLM Model: qwen2.5-7b
✅ Using Foundry SDK Pattern: workshop_utils with FoundryLocalManager
✅ Pre-flight passed: True
✅ Comparison completed: True
✅ Both models responded: True
🎉 ALL CHECKS PASSED! Notebook completed successfully.
   SLM (phi-4-mini) vs LLM (qwen2.5-7b) comparison completed.
   Performance: SLM is 5.14x faster



---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসাধ্য সঠিক অনুবাদের চেষ্টা করি, তবে দয়া করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। নথিটির মূল ভাষায় থাকা সংস্করণটিকে প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ সুপারিশ করা হয়। এই অনুবাদ ব্যবহারের ফলে সৃষ্ট কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যার জন্য আমরা দায়ী নই।
